Das trainieren von komplizierten Modellen geht dank Pytorch genaus so einfach. Unten ist der Code für ein sogenanntes Multi-Layer-Perceptron (MLP) - ein [mehrlagiges neuronales Netzwerk](https://de.wikipedia.org/wiki/Perzeptron).

Ein MLP-Layer besteht aus einer linearen Funktion (in Pytorch `nn.Linear`) und einer nicht-linearen Funktion (zum Beispiel ReLU in Pytorch `nn.ReLU`). Die ReLU Funktion verändert positive Werte nicht d. h. \\(r(x)=x\\). Alle negative Werte jedoch werden auf die 0 Abgebi\
ldet d.h. \\(r(x)=0\\). Die Definition ist: \\(r(x) = \max(0, x)\\).

Allgemein gilt, dass tiefe Modelle flexibler sind und sich besser an die Daten anpassen. Man kann mit solchen Netzwerken sehr komplizierte Probleme lösen. Zum Beispiel können wir mit ähnlichen Netzwerken Bilder von Hunden, den darauf abgebildeten Hundearten zuordnen. Wi\
r müssen jedoch bedenken, dass wir die Netzwerke vorher trainieren müssen und dafür benötigen wir viele Beispielbilder inklusive den dazugehörigen Labels. [Video zu neuronalen Netzwerken](https://www.youtube.com/watch?v=o3RDCSJH2oo)

Finde auch hier passende Werte für die sogenannten Hyperparameter
`n_steps` und `learning_rate`.


In [ ]:
import torch
import numpy as np
import torch.nn as nn

D = np.load('data/train_data.npy')

In [ ]:
def train(D):

    # Hyper-parameter
    n_steps = ...
    learning_rate = ...
    input_size = 13
    output_size = 1

    # Daten vorbereiten
    X = D[:, :-1].astype(np.float32)
    y = D[:, -1].astype(np.float32)
    X = torch.from_numpy(X)
    y = torch.from_numpy(y)
    feature_means = torch.mean(X, dim=0)

    # Modell definieren
    class MLP(nn.Module):
        def __init__(self, input_size):
            super().__init__()
            n_neurons = 4
            self.layers = nn.Sequential(
                nn.Linear(input_size, n_neurons),
                nn.ReLU(),
                nn.Linear(n_neurons, n_neurons),
                nn.ReLU(),
                nn.Linear(n_neurons, output_size)
            )

        def forward(self, x):
            x = x - feature_means
            out = self.layers(x)
            return out

    model = MLP(input_size)

    # loss
    # Dokumentation: https://pytorch.org/docs/stable/nn.html#torch.nn.BCEWithLogitsLoss
    criterion = nn.BCEWithLogitsLoss()  # Sigmoid und Binary-Cross-Entropy-Loss

    # optimizer
    # Dokumentation: https://pytorch.org/docs/stable/optim.html#torch.optim.Adam
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    # Alternative zu Adam
    # Dokumentation: https://pytorch.org/docs/stable/optim.html#torch.optim.SGD
    # momentum = 0.9  # Wert zwischen 0. und 1.
    # optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

    # trainieren des Modells
    for e in range(n_steps):
        # forward pass
        outputs = model.forward(X)[:, 0]
        cost = criterion(outputs, y)

        # backward pass (berechnet die Gradienten automatisch)
        optimizer.zero_grad()  # reset gradients (torch akkumuliert Gradienten)
        cost.backward()  # berechnen der Gradienten

        # Optimierungsschritt durchfuehren
        optimizer.step()

        # Berechnung der Accuracy
        pred_labels = outputs > 0
        is_correct = torch.eq(pred_labels, y.byte()).float()
        accuracy = torch.mean(is_correct).item()

    return model
